In [2]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import models_vit

In [3]:

imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])


def prepare_model(chkpt_dir, arch='vit_large_patch16'):
    # build model
    model = models_vit.__dict__[arch](
        img_size=224,
        num_classes=5,
        drop_path_rate=0,
        global_pool=True,
    )
    # load model
    checkpoint = torch.load(chkpt_dir, map_location='cpu')
    msg = model.load_state_dict(checkpoint['model'], strict=False)
    return model

def run_one_image(img, model):
    
    x = torch.tensor(img)
    x = x.unsqueeze(dim=0)
    x = torch.einsum('nhwc->nchw', x)
    
    x = x.to(device, non_blocking=True)
    latent = model.forward_features(x.float())
    latent = torch.squeeze(latent)
    
    return latent



### Load a pre-trained model

In [4]:
# download pre-trained RETFound 

# chkpt_dir = './RETFound_cfp.pth'
chkpt_dir = './result/checkpoints/RETFound_cfp_weights.pth'
model_ = prepare_model(chkpt_dir, 'vit_large_patch16')

device = torch.device('cuda')
model_.to(device)
print('Model loaded.')


/data1/wangjingtao/softs/miniconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Model loaded.


### Load images and save latent feature

In [10]:
# get image list
data_path = './data/rips/'
img_list = os.listdir(data_path)

name_list = []
feature_list = []
model_.eval()

for i in img_list:
    img = Image.open(os.path.join(data_path, i))
    img = img.resize((224, 224))
    img = np.array(img) / 255.

    assert img.shape == (224, 224, 3)

    # normalize by mean and sd
    # can use customised mean and sd for your data
    img = img - imagenet_mean
    img = img / imagenet_std
    
    latent_feature = run_one_image(img, model_)
    
    name_list.append(i)
    feature_list.append(latent_feature.detach().cpu().numpy())
    

In [8]:
latent_csv = pd.DataFrame({'Name':name_list, 'Latent_feature':feature_list})
latent_csv.to_csv('Feature_latent.csv', index = False, encoding='utf8')